In [5]:
import cv2
import math, sys, os
import matplotlib.pyplot as plt
import numpy as np

from utils import *

####### IO ############

img = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

##### Gaussian noise ##
def gen_gaussian_noise(img_in, mu, sigma, amp):
    return img_in + amp * np.random.normal(mu, sigma, img_in.shape)

## Salt pepper noise ##
def gen_salt_pepper_noise(img_in, prob):
    distribution_map = np.random.uniform(0, 1, img.shape)
    res = np.copy(img_in)
    row, col = img_in.shape
    
    # two-tail test-alike rejection interval
    for i in range(row):
        for j in range(col):
            if distribution_map[i, j] < prob:
                res[i, j] = 0
            elif distribution_map[i, j] > 1 - prob: 
                res[i, j] = 255
                
    return res

def box_filter(img_in, size):
    kernel = []
    for i in range(-size // 2, size // 2):
        for j in range(-size // 2, size // 2):
            kernel.append([i, j])
    
    row, col = img_in.shape
    res = np.zeros(img_in.shape)
    scale = size * size
    
    # conventional gaussian filter processing
    for i in range(row):
        for j in range(col):
            val = 0
            for d in kernel:
                di, dj = d
                if i + di >= 0 and i + di < row \
                and j + dj >= 0 and j + dj < col:
                    val += img_in[i + di, j + dj]
            
            res[i, j] = val / scale
    
    return res

def median_filter(img_in, size):
    kernel = []
    for i in range(-size // 2, size // 2):
        for j in range(-size // 2, size // 2):
            kernel.append([i, j])
    
    row, col = img_in.shape
    res = np.zeros(img_in.shape)
    
    # conventional gaussian filter processing
    for i in range(row):
        for j in range(col):
            vals = []
            for d in kernel:
                di, dj = d
                if i + di >= 0 and i + di < row \
                and j + dj >= 0 and j + dj < col:
                    vals.append(img_in[i + di, j + dj])
            
            res[i, j] = np.median(vals)
    
    return res

def snr(signal, noise, img_name):
    signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    noise = (noise - np.min(noise)) / (np.max(noise) - np.min(noise))
    mu_signal = 0
    power_signal = 0
    mu_noise = 0
    power_noise = 0
    
    assert signal.shape == noise.shape, 'SNR for equal size imagr comparison only'
    row, col = signal.shape
    
    for j in range(col):
        for i in range(row):
            mu_signal = mu_signal + signal[i, j]
    mu_signal = mu_signal / (row * col)

    for j in range(col):
        for i in range(row):
            mu_noise = mu_noise + (noise[i, j] - signal[i, j])
    mu_noise = mu_noise / (row * col)

    for j in range(col):
        for i in range(row):
            power_signal = power_signal + math.pow(signal[i, j] - mu_signal, 2)
    power_signal = power_signal / (row * col)

    for i in range(col):
        for j in range(row):
            power_noise = power_noise +  math.pow(noise[i, j] - signal[i, j] - mu_noise, 2)
    power_noise = power_noise / (row * col)

    res = 20 * math.log(math.sqrt(power_signal) / math.sqrt(power_noise), 10)
    print('SNR of image %s is %f' %(img_name, res))

def main():
    
    if not os.path.exists('problem_a'):
        os.mkdir('problem_a')
    
    if not os.path.exists('problem_b'):
        os.mkdir('problem_b')
        
    if not os.path.exists('problem_c'):
        os.mkdir('problem_c')
        
    if not os.path.exists('problem_d'):
        os.mkdir('problem_d')
        
    if not os.path.exists('problem_e'):
        os.mkdir('problem_e')
        
    ##### Gaussian noise ##
    gn_10 = gen_gaussian_noise(img, 0, 1, 10)
    gn_30 = gen_gaussian_noise(img, 0, 1, 30)
    snr(img, gn_10, 'gn_10')
    snr(img, gn_30, 'gn_30')
    cv2.imwrite('problem_a/lena_gaussian_10.png', gn_10) 
    cv2.imwrite('problem_a/lena_gaussian_30.png', gn_30) 
    
    ## Salt pepper noise ##
    gsp_05 = gen_salt_pepper_noise(img, 0.05)
    gsp_10 = gen_salt_pepper_noise(img, 0.1)
    snr(img, gsp_05, 'gsp_05')
    snr(img, gsp_10, 'gsp_10')
    cv2.imwrite('problem_b/lena_gaussian_05.png', gsp_05) 
    cv2.imwrite('problem_b/lena_gaussian_10.png', gsp_10) 
    
    ###### Box filter #####
    names = ['gn_10', 'gn_30', 'gsp_05', 'gsp_10']
    name_idx = 0
    for img_todo in [gn_10, gn_30, gsp_05, gsp_10]:
        for size in [3, 5]:
            img_name = 'problem_c/lena_' + names[name_idx] + '_box_' + str(size) + '.png'
            img_processed = box_filter(img_todo, size)
            snr(img, img_processed, img_name)
            cv2.imwrite(img_name, img_processed)
        name_idx += 1
    
    ###### Median filter ##
    names = ['gn_10', 'gn_30', 'gsp_05', 'gsp_10']
    name_idx = 0
    for img_todo in [gn_10, gn_30, gsp_05, gsp_10]:
        for size in [3, 5]:
            img_name = 'problem_d/lena_' + names[name_idx] + '_med_' + str(size) + '.png'
            img_processed = median_filter(img_todo, size)
            snr(img, img_processed, img_name)
            cv2.imwrite(img_name, img_processed)
        name_idx += 1
    
    #### open, close #####
    names = ['gn_10', 'gn_30', 'gsp_05', 'gsp_10']
    name_idx = 0

    functions = [opening, closing]
    kernel = [[-2, -1], [-2, 0], [-2, 1],
[-1, -2], [-1, -1], [-1, 0], [-1, 1], [-1, 2],
[0, -2],  [0, -1], [0, 0], [0, 1], [0, 2],
[1, -2],  [1, -1], [1, 0], [1, 1], [1, 2],
          [2, -1], [2, 0], [2, 1]]
    
    for img_todo in [gn_10, gn_30, gsp_05, gsp_10]:
        func_idx = 0
        for func in functions:
            if func_idx == 0:
                img_name = 'problem_e/lena_' + names[name_idx] + '_open_' + str(size) + '.png'
                img_processed = func(img_todo, kernel)
                snr(img, img_processed, img_name)
                cv2.imwrite(img_name, img_processed)
            elif func_idx == 1:
                img_name = 'problem_e/lena_' + names[name_idx] + '_clos_' + str(size) + '.png'
                img_processed = func(img_todo, kernel)
                snr(img, img_processed, img_name)
                cv2.imwrite(img_name, img_processed)
            func_idx += 1
        name_idx += 1

if __name__ == '__main__':
    main()
    print('finished all')
    

SNR of image gn_10 is 12.489506
SNR of image gn_30 is 4.984327
SNR of image gsp_05 is 1.868454
SNR of image gsp_10 is -1.037364
SNR of image problem_c/lena_gn_10_box_3.png is 11.137551
SNR of image problem_c/lena_gn_10_box_5.png is 11.237070
SNR of image problem_c/lena_gn_30_box_3.png is 9.424313
SNR of image problem_c/lena_gn_30_box_5.png is 10.545494
SNR of image problem_c/lena_gsp_05_box_3.png is 7.630154
SNR of image problem_c/lena_gsp_05_box_5.png is 9.303153
SNR of image problem_c/lena_gsp_10_box_3.png is 5.516857
SNR of image problem_c/lena_gsp_10_box_5.png is 7.465335
SNR of image problem_d/lena_gn_10_med_3.png is 11.270961
SNR of image problem_d/lena_gn_10_med_5.png is 11.562366
SNR of image problem_d/lena_gn_30_med_3.png is 8.503938
SNR of image problem_d/lena_gn_30_med_5.png is 10.746141
SNR of image problem_d/lena_gsp_05_med_3.png is 10.909508
SNR of image problem_d/lena_gsp_05_med_5.png is 11.036893
SNR of image problem_d/lena_gsp_10_med_3.png is 10.121708
SNR of image pro

In [3]:
import cv2
import numpy as np
import math

def snr(signal, noise, img_name):
    signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    noise = (noise - np.min(noise)) / (np.max(noise) - np.min(noise))
    mu_signal = 0
    power_signal = 0
    mu_noise = 0
    power_noise = 0
    
    assert signal.shape == noise.shape, 'SNR for equal size imagr comparison only'
    row, col = signal.shape
    
    for j in range(col):
        for i in range(row):
            mu_signal = mu_signal + signal[i, j]
    mu_signal = mu_signal / (row * col)

    for j in range(col):
        for i in range(row):
            mu_noise = mu_noise + (noise[i, j] - signal[i, j])
    mu_noise = mu_noise / (row * col)

    for j in range(col):
        for i in range(row):
            power_signal = power_signal + math.pow(signal[i, j] - mu_signal, 2)
    power_signal = power_signal / (row * col)

    for i in range(col):
        for j in range(row):
            power_noise = power_noise +  math.pow(noise[i, j] - signal[i, j] - mu_noise, 2)
    power_noise = power_noise / (row * col)

    res = 20 * math.log(math.sqrt(power_signal) / math.sqrt(power_noise), 10)
    print('SNR of image %s is %f' %(img_name, res))
    
img = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
ta = cv2.imread('snr_test.bmp', cv2.IMREAD_GRAYSCALE)
snr(img, ta, 'snr_test')

SNR of image snr_test is 14.657314
